In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import cv2
import numpy as np
from ROIPoolingLayer import RoiPoolingConv
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import roi_helpers


print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Num GPUs Available:  2


In [2]:
import voc_parser
import data_augment
import data_generators
import resnet
import config
import random
from tensorflow.keras.layers import Input

C = config.Config()
C.num_rois = 32

dataPath = 'dataset/voc2012/VOCdevkit'

all_imgs, classes_count, class_mapping = voc_parser.get_data(dataPath)


Processing 2012_003196.xml:   1%|          | 130/17125 [00:00<00:13, 1299.79it/s]

Parsing annotation files


Processing 2010_000111.xml:  17%|█▋        | 2877/17125 [00:02<00:12, 1096.03it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Processing 2010_001773.xml:  45%|████▍     | 7646/17125 [00:05<00:06, 1556.76it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Processing 2010_002899.xml:  75%|███████▍  | 12826/17125 [00:08<00:02, 1446.45it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in 

In [3]:
import tensorflow.keras.backend as K

if 'bg' not in classes_count:
    classes_count['bg'] = 0
    class_mapping['bg'] = len(class_mapping)

C.class_mapping = class_mapping

inv_map = {v: k for k, v in class_mapping.items()}

random.shuffle(all_imgs)

num_imgs = len(all_imgs)

train_imgs = [s for s in all_imgs if s['imageset'] == 'train']
val_imgs = [s for s in all_imgs if s['imageset'] == 'val']
test_imgs = [s for s in all_imgs if s['imageset'] == 'test']

print('Num train samples {}'.format(len(train_imgs)))
print('Num val samples {}'.format(len(val_imgs)))
print('Num test samples {}'.format(len(test_imgs)))

data_gen_train = data_generators.get_anchor_gt(
    train_imgs, classes_count, C, resnet.get_img_output_length, K.image_data_format(), mode='train')
data_gen_val = data_generators.get_anchor_gt(
    val_imgs, classes_count, C, resnet.get_img_output_length, K.image_data_format(), mode='val')
data_gen_test = data_generators.get_anchor_gt(
    test_imgs, classes_count, C, resnet.get_img_output_length, K.image_data_format(), mode='val')

Num train samples 5717
Num val samples 5823
Num test samples 0


In [4]:
input_shape_img = (None, None, 3)

# input placeholder 정의
img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(None, 4))

shared_layers = resnet.nn_base(img_input, trainable=True)

num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn = resnet.rpn(shared_layers, num_anchors)

classifier = resnet.classifier(shared_layers, roi_input, C.num_rois, nb_classes=len(
    classes_count), trainable=True)

model_rpn = Model(img_input, rpn[:2])
model_classifier = Model([img_input, roi_input], classifier)

model_all = Model([img_input, roi_input], rpn[:2] + classifier)


channels_last


In [5]:
import losses
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

C.base_net_weights = resnet.get_weight_path()
try:
    print(C.base_net_weights)
    # load_weights by name
    # some keras application model does not containing name
    # for this kinds of model, we need to re-construct model with naming
    print('loading weights from {}'.format(C.base_net_weights))
    model_rpn.load_weights(C.base_net_weights, by_name=True)
    model_classifier.load_weights(C.base_net_weights, by_name=True)
except:
    print('Could not load pretrained model weights. Weights can be found in the keras application folder \
        https://github.com/fchollet/keras/tree/master/keras/applications')
    

optimizer = Adam(lr=1e-5)
optimizer_classifier = Adam(lr=1e-5)
model_rpn.compile(optimizer=optimizer, loss=[losses.rpn_loss_cls(
    num_anchors), losses.rpn_loss_regr(num_anchors)])
model_classifier.compile(optimizer=optimizer_classifier, loss=[losses.class_loss_cls, losses.class_loss_regr(
    len(classes_count)-1)], metrics={'dense_class_{}'.format(len(classes_count)): 'accuracy'})
model_all.compile(optimizer='sgd', loss='mae')



pretrained/resnet50_weights_tf_dim_ordering_tf_kernels.h5
loading weights from pretrained/resnet50_weights_tf_dim_ordering_tf_kernels.h5


In [6]:
import time
epoch_length = 1000
num_epochs = 20
iter_num = 0
train_step = 0

losses = np.zeros((epoch_length, 5))
rpn_accuracy_rpn_monitor = []
rpn_accuracy_for_epoch = []
start_time = time.time()

best_loss = np.Inf

class_mapping_inv = {v: k for k, v in class_mapping.items()}
print('Starting training')

Starting training


In [7]:
from tensorflow.python.keras.utils import generic_utils
for epoch_num in range(num_epochs):

    progbar = generic_utils.Progbar(epoch_length)   # keras progress bar 사용
    print('Epoch {}/{}'.format(epoch_num + 1, num_epochs))

    while True:
        # try:
        # mean overlapping bboxes 출력
        if len(rpn_accuracy_rpn_monitor) == epoch_length and C.verbose:
            mean_overlapping_bboxes = float(
                sum(rpn_accuracy_rpn_monitor))/len(rpn_accuracy_rpn_monitor)
            rpn_accuracy_rpn_monitor = []
            print('Average number of overlapping bounding boxes from RPN = {} for {} previous iterations'.format(
                mean_overlapping_bboxes, epoch_length))
            if mean_overlapping_bboxes == 0:
                print(
                    'RPN is not producing bounding boxes that overlap the ground truth boxes. Check RPN settings or keep training.')

        # data generator에서 X, Y, image 가져오기
        X, Y, img_data = next(data_gen_train)

        loss_rpn = model_rpn.train_on_batch(X, Y)
        # write_log(callback, ['rpn_cls_loss',
        #                      'rpn_reg_loss'], loss_rpn, train_step)

        P_rpn = model_rpn.predict_on_batch(X)

        R = roi_helpers.rpn_to_roi(P_rpn[0], P_rpn[1], C, K.image_data_format(
        ), use_regr=True, overlap_thresh=0.7, max_boxes=300)
        # note: calc_iou converts from (x1,y1,x2,y2) to (x,y,w,h) format
        X2, Y1, Y2, IouS = roi_helpers.calc_iou(R, img_data, C, class_mapping)

        if X2 is None:
            rpn_accuracy_rpn_monitor.append(0)
            rpn_accuracy_for_epoch.append(0)
            continue

        # sampling positive/negative samples
        neg_samples = np.where(Y1[0, :, -1] == 1)
        pos_samples = np.where(Y1[0, :, -1] == 0)

        if len(neg_samples) > 0:
            neg_samples = neg_samples[0]
        else:
            neg_samples = []

        if len(pos_samples) > 0:
            pos_samples = pos_samples[0]
        else:
            pos_samples = []

        rpn_accuracy_rpn_monitor.append(len(pos_samples))
        rpn_accuracy_for_epoch.append((len(pos_samples)))

        if C.num_rois > 1:
            if len(pos_samples) < C.num_rois//2:
                selected_pos_samples = pos_samples.tolist()
            else:
                selected_pos_samples = np.random.choice(
                    pos_samples, C.num_rois//2, replace=False).tolist()
            try:
                selected_neg_samples = np.random.choice(
                    neg_samples, C.num_rois - len(selected_pos_samples), replace=False).tolist()
            except:
                selected_neg_samples = np.random.choice(
                    neg_samples, C.num_rois - len(selected_pos_samples), replace=True).tolist()

            sel_samples = selected_pos_samples + selected_neg_samples
        else:
            # in the extreme case where num_rois = 1, we pick a random pos or neg sample
            selected_pos_samples = pos_samples.tolist()
            selected_neg_samples = neg_samples.tolist()
            if np.random.randint(0, 2):
                sel_samples = random.choice(neg_samples)
            else:
                sel_samples = random.choice(pos_samples)

        loss_class = model_classifier.train_on_batch(
            [X, X2[:, sel_samples, :]], [Y1[:, sel_samples, :], Y2[:, sel_samples, :]])
        # write_log(callback, ['detection_cls_loss', 'detection_reg_loss',
        #                      'detection_acc'], loss_class, train_step)
        train_step += 1

        losses[iter_num, 0] = loss_rpn[1]
        losses[iter_num, 1] = loss_rpn[2]

        losses[iter_num, 2] = loss_class[1]
        losses[iter_num, 3] = loss_class[2]
        losses[iter_num, 4] = loss_class[3]

        iter_num += 1

        progbar.update(iter_num, [('rpn_cls', np.mean(losses[:iter_num, 0])), ('rpn_regr', np.mean(losses[:iter_num, 1])),
                                  ('detector_cls', np.mean(losses[:iter_num, 2])), ('detector_regr', np.mean(losses[:iter_num, 3]))])

        if iter_num == epoch_length:
            loss_rpn_cls = np.mean(losses[:, 0])
            loss_rpn_regr = np.mean(losses[:, 1])
            loss_class_cls = np.mean(losses[:, 2])
            loss_class_regr = np.mean(losses[:, 3])
            class_acc = np.mean(losses[:, 4])

            mean_overlapping_bboxes = float(
                sum(rpn_accuracy_for_epoch)) / len(rpn_accuracy_for_epoch)
            rpn_accuracy_for_epoch = []

            if C.verbose:
                print('Mean number of bounding boxes from RPN overlapping ground truth boxes: {}'.format(
                    mean_overlapping_bboxes))
                print(
                    'Classifier accuracy for bounding boxes from RPN: {}'.format(class_acc))
                print('Loss RPN classifier: {}'.format(loss_rpn_cls))
                print('Loss RPN regression: {}'.format(loss_rpn_regr))
                print('Loss Detector classifier: {}'.format(loss_class_cls))
                print('Loss Detector regression: {}'.format(loss_class_regr))
                print('Elapsed time: {}'.format(time.time() - start_time))

            curr_loss = loss_rpn_cls + loss_rpn_regr + loss_class_cls + loss_class_regr
            iter_num = 0
            start_time = time.time()

            # write_log(callback,
            #           ['Elapsed_time', 'mean_overlapping_bboxes', 'mean_rpn_cls_loss', 'mean_rpn_reg_loss',
            #            'mean_detection_cls_loss', 'mean_detection_reg_loss', 'mean_detection_acc', 'total_loss'],
            #           [time.time() - start_time, mean_overlapping_bboxes, loss_rpn_cls, loss_rpn_regr,
            #            loss_class_cls, loss_class_regr, class_acc, curr_loss],
            #           epoch_num)

            if curr_loss < best_loss:
                if C.verbose:
                    print('Total loss decreased from {} to {}, saving weights'.format(
                        best_loss, curr_loss))
                best_loss = curr_loss
                model_all.save_weights(C.model_path)

            break

        # except Exception as e:
        #     print('Exception: {}'.format(e))
        #     continue

print('Training complete, exiting.')


Epoch 1/20
1000/1000 [==============================] - 548s 548ms/step - rpn_cls: 3.4331 - rpn_regr: 0.2169 - detector_cls: 1.5052 - detector_regr: 0.4388
Mean number of bounding boxes from RPN overlapping ground truth boxes: 13.871128871128871
Classifier accuracy for bounding boxes from RPN: 0.670125
Loss RPN classifier: 2.942597916045732
Loss RPN regression: 0.2199396117674187
Loss Detector classifier: 1.2343699016720056
Loss Detector regression: 0.43301878434419633
Elapsed time: 548.1032915115356
Total loss decreased from inf to 4.829926213829352, saving weights
Epoch 2/20
1000/1000 [==============================] - 449s 449ms/step - rpn_cls: 2.4098 - rpn_regr: 0.2013 - detector_cls: 0.9111 - detector_regr: 0.4092
Mean number of bounding boxes from RPN overlapping ground truth boxes: 14.135
Classifier accuracy for bounding boxes from RPN: 0.70103125
Loss RPN classifier: 2.420644920081746
Loss RPN regression: 0.2103866858743131
Loss Detector classifier: 0.8694406774691306
Loss Dete

1000/1000 [==============================] - 412s 412ms/step - rpn_cls: 1.4279 - rpn_regr: 0.1677 - detector_cls: 0.6383 - detector_regr: 0.2914
Mean number of bounding boxes from RPN overlapping ground truth boxes: 17.931
Classifier accuracy for bounding boxes from RPN: 0.755875
Loss RPN classifier: 1.4290908598597674
Loss RPN regression: 0.16032189589878545
Loss Detector classifier: 0.6437116477251984
Loss Detector regression: 0.28728459648694843
Elapsed time: 412.6073033809662
Total loss decreased from 2.548783592723974 to 2.5204089999707, saving weights
Epoch 12/20
1000/1000 [==============================] - 422s 422ms/step - rpn_cls: 1.3540 - rpn_regr: 0.1642 - detector_cls: 0.5962 - detector_regr: 0.2816
Mean number of bounding boxes from RPN overlapping ground truth boxes: 19.025
Classifier accuracy for bounding boxes from RPN: 0.77471875
Loss RPN classifier: 1.2606047067423398
Loss RPN regression: 0.1651038330476731
Loss Detector classifier: 0.5849500677268952
Loss Detector re

In [8]:
model_classifier.outputs[1]

<tf.Tensor 'dense_regress_21/Identity:0' shape=(1, 32, 80) dtype=float32>

In [15]:
import pickle
with open('config.pickle', 'wb') as config_f:
    pickle.dump(C, config_f)
